In [1]:
%%HTML
<style>
.container { width:100% }
</style>

## Import der benötigten Bibliotheken

In [2]:
import pprint
import nltk
import csv
import operator
import re
import pandas    as pd
from pymongo     import MongoClient
from nltk        import word_tokenize
from nltk.corpus import stopwords
from string      import ascii_letters
from pathlib     import Path

## Laden der Sätze und falls notwendig konvertierung der .txt-Datenquellen in .csv-Dateien

In [3]:
if Path('sentences_de.csv').is_file():
    sentences_csv = pd.read_csv ('sentences_de.csv', header = None)
elif Path('deu_news_2015_3M-sentences.txt').is_file():
    sentences_csv = pd.read_csv ('deu_news_2015_3M-sentences.txt', header = None, delimiter='\t')
    sentences_csv.to_csv ('sentences_de.csv', index=None)
else:
    print("Die benötigten Daten existieren nicht. Bitte lade sie von folgender Seite runter: https://www.kaggle.com/rtatman/3-million-german-sentences")

## Laden der Stopwords

In [4]:
nltk.download('stopwords')
stop = set(stopwords.words('german'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Luisa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Entfernen der Stopwords und allen Zeichen, die keine Ascii-Buchstaben sind

In [5]:
def remove_stopwords_and_nonascii(sentences):
    queue = []
    for sentence in sentences:
        just_ascii = re.sub(r"[^{}]".format(ascii_letters + 'äöüÄÖÜß'), ' ', sentence) 
        result = ' '.join([i for i in just_ascii.lower().split() if i not in stop and i != ' '])
        if result != ' ': queue.append(result)
    return queue

In [6]:
nostop = remove_stopwords_and_nonascii(sentences_csv[1]) #(sentences_csv[1][100:200]) Für Test nur 100 Datensätze verwenden
df = pd.DataFrame(nostop)
df.to_csv('sentences_preprocessed.csv', index=False)

## Verbindung mit mongoDB

In [7]:
connection_string      = "mongodb://localhost:27017"
db                     = MongoClient(connection_string)
sentences              = db.dw.sentences
sentences_preprocessed = db.dw.sentences_preprocessed

## Sentences und Stopwords der Database in mongoDB hinzufügen

`read_csv()` liest eine .csv-Datei ein und gibt ein Dictionary zurück, dass dann mit `save_to_mongo()` in einer Collection gespeichert wird. Dafür kann zunächst die Collection gelöscht werden (`drop_collection = True`). Sonst werden die Daten zusätzlich in die Collection eingefügt.

In [8]:
def read_csv(path, columns):
    reader = open(path, 'r', encoding='utf-8')
    return csv.DictReader(reader, columns)   

In [9]:
def save_to_mongo(collection, path, columns, drop_collection):
    if drop_collection:
        collection.drop()
    data = read_csv(path, columns)
    result = collection.insert_many(data)
    print('%d rows are saved to "%s" collection successfully!' % (len(result.inserted_ids), collection.name))

In [10]:
save_to_mongo(sentences, 'sentences_de.csv', ('id','sentence'), True)

2646184 rows are saved to "sentences" collection successfully!


In [11]:
save_to_mongo(sentences_preprocessed, 'sentences_preprocessed.csv', ('s'), True)

2646185 rows are saved to "sentences_preprocessed" collection successfully!


Wenn ich die als Column-Name gleich `sentence` genommen habe, dann hat er daraus eine Spalte `s`, eine `e`, eine `n` und so weiter gemacht...Deswegen hier mit der späteren Umbenennung.

In [12]:
sentences_preprocessed.update_many( {}, { '$rename': { "s": "sentence" } } )

In [13]:
def count_bigrams(data):
    bigram_dict = dict()
    for text in data:
        bigrams = list(nltk.bigrams(text.split()))
        for bigram in bigrams:
            if bigram in bigram_dict:
                bigram_dict[bigram] += 1
            else:
                bigram_dict[bigram] = 1
    return bigram_dict

In [14]:
def get_k_most_frequent(dictionary, k):
    sorted_dictionary =  dict(sorted(dictionary.items(), key=operator.itemgetter(1),reverse=True))
    result = sorted_dictionary.items()
    return list(result)[:k]

In [15]:
def pretty_print_most_frequent(data):
    for i in range(len(data)):
        words, frequency = data[i]
        print('Platz ', i+1, ': Die Wortkombination "', ' '.join(words), \
              '" kommt in den Daten ', frequency, ' mal vor.', sep='')

In [16]:
def get_k_most_frequent_bigrams(data, k):
    bigram_dict   = count_bigrams(data)
    most_frequent = get_k_most_frequent(bigram_dict, k)
    pretty_print_most_frequent(most_frequent)

In [17]:
get_k_most_frequent_bigrams(nostop, 10)

Platz 1: Die Wortkombination "e mail" kommt in den Daten 15108 mal vor.
Platz 2: Die Wortkombination "millionen euro" kommt in den Daten 11995 mal vor.
Platz 3: Die Wortkombination "milliarden euro" kommt in den Daten 8993 mal vor.
Platz 4: Die Wortkombination "antwort schreiben" kommt in den Daten 7553 mal vor.
Platz 5: Die Wortkombination "seit jahren" kommt in den Daten 6488 mal vor.
Platz 6: Die Wortkombination "new york" kommt in den Daten 5108 mal vor.
Platz 7: Die Wortkombination "us dollar" kommt in den Daten 4957 mal vor.
Platz 8: Die Wortkombination "mail adresse" kommt in den Daten 4883 mal vor.
Platz 9: Die Wortkombination "orf at" kommt in den Daten 4797 mal vor.
Platz 10: Die Wortkombination "vergangenen jahr" kommt in den Daten 4776 mal vor.


## ToDos: 
- Einfügen von sentences_preprocessed in DB ohne Umweg über DataFrame und .csv
- Spaltenname ohne Umbenennung
- Daten für Analyse aus DB ziehen
- Das Ergebnis (komplettes Bigram-Dict?) in der DB speichern
- Weitere Analysen?